In [ ]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import Range1d
output_notebook()

import numpy as np

import json
import csv
import os
import numpy as np

import voltagebudget

from scipy.optimize import least_squares

from voltagebudget.neurons import adex
from voltagebudget.neurons import shadow_adex

from voltagebudget.util import poisson_impulse
from voltagebudget.util import read_results
from voltagebudget.util import read_stim
from voltagebudget.util import read_args
from voltagebudget.util import read_modes

from voltagebudget.util import select_n
from voltagebudget.util import create_times
from voltagebudget.util import filter_voltages
from voltagebudget.util import filter_spikes
from voltagebudget.util import budget_window
from voltagebudget.util import locate_firsts
from voltagebudget.util import locate_peaks
from voltagebudget.util import find_E
from voltagebudget.util import find_phis

In [ ]:
stim = "../data/stim1.csv"
target=0
E_0=0
N=250
t=0.4
A=0.05e-9
Z_0=1e-6
Z_max=1
f=8
n_jobs=1
mode='regular'
noise=False
no_lock=False
verbose=True
seed_value=42
"""Find the optimal Z value for a given (A, f)."""

np.random.seed(seed_value)

# --------------------------------------------------------------
# Temporal params
time_step = 1e-5

# ---------------------------------------------------------------
if verbose:
    print(">>> Setting mode.")

params, w_in, bias_in, sigma = read_modes(mode)
if not noise:
    sigma = 0

# ---------------------------------------------------------------
if verbose:
    print(">>> Importing stimulus from {}.".format(stim))

stim_data = read_stim(stim)
ns = np.asarray(stim_data['ns'])
ts = np.asarray(stim_data['ts'])

# ---------------------------------------------------------------
if verbose:
    print(">>> Creating reference spikes.")

ns_ref, ts_ref, voltages_ref = adex(
    N,
    t,
    ns,
    ts,
    w_in=w_in,
    bias_in=bias_in,
    f=0.0,
    A=0,
    phi=0,
    sigma=sigma,
    budget=True,
    save_args=None,
    time_step=time_step,
    seed_value=seed_value,
    **params)

if ns_ref.size == 0:
    raise ValueError("The reference model didn't spike.")

# --------------------------------------------------------------
# Find T, E and phis
T = 1 / float(f)  # Analysis window
E = find_E(E_0, ns_ref, ts_ref, no_lock=no_lock, verbose=verbose)
_, phi_E = find_phis(E, f, 0, verbose=verbose)

# Filter ref spikes into the window of interest
ns_ref, ts_ref = filter_spikes(ns_ref, ts_ref, (E, E + T))
if verbose:
    print(">>> {} spikes in the analysis window.".format(ns_ref.size))

# ---------------------------------------------------------------
Z = 0
bias = bias_in  - (Z * A)

ns_y, ts_y, voltages_y = adex(
    N,
    t,
    ns,
    ts,
    E=E,
    n_cycles=2,
    w_in=w_in,
    bias_in=bias,
    f=f,
    A=A,
    phi=phi_E,
    sigma=sigma,
    budget=True,
    save_args=None,
    time_step=time_step,
    seed_value=seed_value,
    **params)

ns_y, ts_y = filter_spikes(ns_y, ts_y, (E, E + T))

delta = float(abs(ts_ref.size - ts_y.size)) / N
loss = abs(target - delta)

if verbose:
    print("(Z {:0.18f}, bias_adj/bias {:0.18f}) -> (loss {:0.6f})".
          format(Z, bias / bias_in, loss))

In [ ]:
times = create_times((0, t), time_step)

x = voltages_y["V_m"].T

# -
p = figure(plot_width=600, plot_height=300)
# p.line(x=E, y=np.linspace(x.min(), x.max(), 10), color="purple")
# p.line(x=times, y=x.mean(1), color='red', alpha=0.8)

for n in range(10):
    p.line(x=times, y=x[:, n], color="black", alpha=0.1)
    
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'V_comp (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
# p.x_range = Range1d(0.2, 0.24)
show(p)

In [ ]:
ts_y.size